In [ ]:
from vectorrvnn.data import *
from vectorrvnn.utils import *
from vectorrvnn.baselines import *
from more_itertools import unzip
import svgpathtools as svg
from tqdm import tqdm

data = TripletDataset('../data/MikeAnnotations')

In [ ]:
def getAnnotationsByName (name) : 
    ann = []
    for i, d in enumerate(data) :
        try : 
            id, _name_ = data.metadata[i].split(', ')
            if _name_ == name : 
                ann.append((id, d))
        except Exception :
            pass
    return ann

In [ ]:
mike = getAnnotationsByName('mike')
sumit = getAnnotationsByName('sumit')

commonIds = set(unzip(mike)[0]).intersection(set(unzip(sumit)[0]))

mike = dict([(id, d) for id, d in mike if id in commonIds])
sumit = dict([(id, d) for id, d in sumit if id in commonIds])

# will be the same as sumit's
svgFiles = list(map(lambda x : mike[x].svgFile, commonIds))
docs = dict(zip(commonIds, map(svg.Document, svgFiles)))

auto = dict([(id, autogroup(docs[id])) for id in tqdm(commonIds)])
sug = dict([(id, suggero(docs[id])) for id in tqdm(commonIds)])


In [ ]:
def printFMI (d1, d2) : 
    print("FMI-1:", avg(map(lambda id : fmi(d1[id], d2[id], 1), commonIds)))
    print("FMI-2:", avg(map(lambda id : fmi(d1[id], d2[id], 2), commonIds)))    
    print("FMI-3:", avg(map(lambda id : fmi(d1[id], d2[id], 3), commonIds)))   

print("Mike v Sumit")
printFMI(mike, sumit)
print("Mike v Autogroup")
printFMI(mike, auto)
print("Mike v Suggero")
printFMI(mike, sug)
print("Sumit v Autogroup")
printFMI(sumit, auto)
print("Sumit v Suggero")
printFMI(sumit, sug)
print("Autogroup v Suggero")
printFMI(sug, auto)


In [ ]:
def avgNumberOfNodesAtLevel (data, level) :
    avgNodes = []
    for id, d in data.items() : 
        d_ = forest2tree(d)
        setNodeDepths(d_)
        avgNodes.append(len([n for n in d_.nodes if d_.nodes[n]['depth'] == level]))
    return avg(avgNodes)

print(avgNumberOfNodesAtLevel(mike, 3))
print(avgNumberOfNodesAtLevel(sumit, 3))
print(avgNumberOfNodesAtLevel(auto, 3))
print(avgNumberOfNodesAtLevel(sug, 3))
        